In [1]:
from pathlib import Path
import polars as pl
from datasets import Dataset
import numpy as np
from autofaiss import build_index
from FlagEmbedding import BGEM3FlagModel
from core_pro.ultilities import make_dir
import sys
sys.path.extend([str(Path.home() / 'PycharmProjects/item_matching')])

from notebooks.benchmark.data_load import load

In [2]:
df, col, path = load()
df.head()

Data Shape: (72110, 6)


id,q_item_id,q_level1_global_be_category,q_item_name,q_link_first_image,q_item_name_clean
u32,i64,str,str,str,str
0,4992244030,"""Men Shoes""","""Hai Miếng Lót Đệm Giày Mềm Mã …","""https://cf.shopee.sg/file/bfe9…","""hai miếng lót đệm giày mềm mã …"
1,6655855608,"""Men Shoes""","""Ủng bảo hộ lao động nam Jogger…","""https://cf.shopee.sg/file/13e3…","""ủng bảo hộ lao động nam jogger…"
2,22013169719,"""Men Shoes""","""Giày lười nam - Slip on nam vả…","""https://cf.shopee.sg/file/sg-1…","""giày lười nam - slip on nam vả…"
3,25414409766,"""Men Shoes""","""Giày Nike_AF1 Xám Lông Chuột, …","""https://cf.shopee.sg/file/vn-1…","""giày nike_af1 xám lông chuột, …"
4,23275720358,"""Men Shoes""","""Dép DG Nam Phối Dây Chéo Cực Đ…","""https://cf.shopee.sg/file/vn-1…","""dép dg nam phối dây chéo cực đ…"


In [3]:
path_tmp_array = Path('tmp/array')
path_tmp_ds = Path('tmp/ds')
make_dir(path_tmp_ds)
make_dir(path_tmp_array)

file_embed = path_tmp_array / 'embed.npy'
if not file_embed.exists():
    model = BGEM3FlagModel('BAAI/bge-m3', use_fp16=False)
    embeddings = model.encode(
        df['q_item_name_clean'].to_list(),
        batch_size=8,
        max_length=80,
        return_dense=True,
        return_sparse=False,
        return_colbert_vecs=False
    )['dense_vecs']
    np.save(file_embed, embeddings)
else:
    embeddings = np.load(file_embed)
print(embeddings.shape)

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

/home/kevin/miniforge3/envs/item/lib/python3.12/site-packages/FlagEmbedding/BGE_M3/modeling.py:335: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  colbert_state_dict = torch.

(72110, 1024)


In [4]:
df = df.with_columns(pl.Series(values=embeddings, name='embed'))
dataset = Dataset.from_polars(df)
dataset.set_format(type='numpy', columns=['embed'], output_all_columns=True)

In [7]:
path_index = Path('tmp/index')
build_index(
    embeddings=embeddings,
    index_path=str(path_index / f'ip.index'),
    index_infos_path=str(path_index / f'index.json'),
    save_on_disk=True,
    metric_type='ip',
    verbose=30,
)

  0%|          | 0/1 [00:00<?, ?it/s]


(<faiss.swigfaiss_avx2.IndexHNSWFlat; proxy of <Swig Object of type 'faiss::IndexHNSWFlat *' at 0x75cac2117240> >,
 {'index_key': 'HNSW32',
  'index_param': 'efSearch=866',
  'index_path': 'tmp/index/ip.index',
  'size in bytes': 314987402,
  'avg_search_speed_ms': 9.663053585544278,
  '99p_search_speed_ms': 16.96898969617905,
  'reconstruction error %': 0.0,
  'nb vectors': 72110,
  'vectors dimension': 1024,
  'compression ratio': 0.9376964225381941})

In [8]:
# add index
dataset.load_faiss_index('embed', path_index / f'ip.index')

In [11]:
score, result = dataset.get_nearest_examples_batch(
    'embed',
    np.asarray(dataset['embed']),
    k=5
)

dict_ = {'score': [list(i) for i in score]}
df_score = pl.DataFrame(dict_)
df_result = (
    pl.DataFrame(result).drop(['embed'])
    .select(pl.all().name.prefix(f'db_'))
)

In [12]:
df_match = pl.concat([df, df_result, df_score], how='horizontal')
col_explode = [i for i in df_match.columns if 'notebooks' in i] + ['score']
df_match = df_match.explode(col_explode)

In [13]:
df_match

id,q_item_id,q_level1_global_be_category,q_item_name,q_link_first_image,q_item_name_clean,embed,db_id,db_q_item_id,db_q_level1_global_be_category,db_q_item_name,db_q_link_first_image,db_q_item_name_clean,score
u32,i64,str,str,str,str,"array[f32, 1024]",i64,i64,str,str,str,str,f32
0,4992244030,"""Men Shoes""","""Hai Miếng Lót Đệm Giày Mềm Mã …","""https://cf.shopee.sg/file/bfe9…","""hai miếng lót đệm giày mềm mã …","[-0.033611, 0.016315, … 0.003427]",0,4992244030,"""Men Shoes""","""Hai Miếng Lót Đệm Giày Mềm Mã …","""https://cf.shopee.sg/file/bfe9…","""hai miếng lót đệm giày mềm mã …",1.0
0,4992244030,"""Men Shoes""","""Hai Miếng Lót Đệm Giày Mềm Mã …","""https://cf.shopee.sg/file/bfe9…","""hai miếng lót đệm giày mềm mã …","[-0.033611, 0.016315, … 0.003427]",11781,11812317752,"""Men Shoes""","""Cặp Lót Giày Mềm ( Chân Phải +…","""https://cf.shopee.sg/file/acd1…","""cặp lót giày mềm mã l03""",0.814686
0,4992244030,"""Men Shoes""","""Hai Miếng Lót Đệm Giày Mềm Mã …","""https://cf.shopee.sg/file/bfe9…","""hai miếng lót đệm giày mềm mã …","[-0.033611, 0.016315, … 0.003427]",13838,17237055021,"""Men Shoes""","""Cặp Lót Giày Mềm ( Chân Phải +…","""https://cf.shopee.sg/file/vn-1…","""cặp lót giày mềm mềm mại""",0.787536
0,4992244030,"""Men Shoes""","""Hai Miếng Lót Đệm Giày Mềm Mã …","""https://cf.shopee.sg/file/bfe9…","""hai miếng lót đệm giày mềm mã …","[-0.033611, 0.016315, … 0.003427]",36429,852601616,"""Men Shoes""","""2 MIẾNG LÓT Gót GIÀY TĂNG CHIỀ…","""https://cf.shopee.sg/file/sg-1…","""2 miếng lót gót giày tăng chiề…",0.770263
0,4992244030,"""Men Shoes""","""Hai Miếng Lót Đệm Giày Mềm Mã …","""https://cf.shopee.sg/file/bfe9…","""hai miếng lót đệm giày mềm mã …","[-0.033611, 0.016315, … 0.003427]",21480,25318428920,"""Men Shoes""","""02 Miếng Lót Giày Độn Đế Tăng …","""https://cf.shopee.sg/file/vn-1…","""02 miếng lót giày độn đế tăng …",0.763561
…,…,…,…,…,…,…,…,…,…,…,…,…,…
72109,18432181296,"""Men Shoes""","""Giày Thể Thao Sneaker Nam Nữ 𝗝…","""https://cf.shopee.sg/file/5339…","""giày thể thao sneaker nam nữ 𝗝…","[-0.008872, -0.014089, … 0.00096]",72109,18432181296,"""Men Shoes""","""Giày Thể Thao Sneaker Nam Nữ 𝗝…","""https://cf.shopee.sg/file/5339…","""giày thể thao sneaker nam nữ 𝗝…",1.0
72109,18432181296,"""Men Shoes""","""Giày Thể Thao Sneaker Nam Nữ 𝗝…","""https://cf.shopee.sg/file/5339…","""giày thể thao sneaker nam nữ 𝗝…","[-0.008872, -0.014089, … 0.00096]",18569,17379360669,"""Men Shoes""","""Giày Thể Thao Nam Nữ 𝗝𝗼𝗿𝗱𝗮𝗻 Xa…","""https://cf.shopee.sg/file/e392…","""giày thể thao nam nữ 𝗝𝗼𝗿𝗱𝗮𝗻 xa…",0.960821
72109,18432181296,"""Men Shoes""","""Giày Thể Thao Sneaker Nam Nữ 𝗝…","""https://cf.shopee.sg/file/5339…","""giày thể thao sneaker nam nữ 𝗝…","[-0.008872, -0.014089, … 0.00096]",16553,19623518942,"""Men Shoes""","""Giày Thể Thao Sneaker 𝗝𝗼𝗿𝗱𝗮𝗻 T…","""https://cf.shopee.sg/file/757e…","""giày thể thao sneaker 𝗝𝗼𝗿𝗱𝗮𝗻 t…",0.95498
